In [3]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from climada.util.constants import SYSTEM_DIR

### Table 2 and 3

Create summary table of 100-yr exceedance intensity estimates for ERA5, GCM-baseline, and GCM future runs for different emission scenarios (ssp245, ssp370, ssp585) and both future periods (fut1 = 2041-2060, fut2 = 2081-2100).

In [14]:
tcgi = "SD"

# --- 1. Read raw GCM min/max table for CRH ---
df = pd.read_csv(f'/Users/simonameiler/Documents/work/03_code/repos/CHAZ-hazard-maps/CHAZ-hazard-maps/data/rp100_gcm_minmaxmed_{tcgi}.csv', index_col=0, header=[0,1])

# --- 2. Collapse all base_* → 'base' & strip f'_{tcgi}' suffix ---
new_cols = []
for scen, stat in df.columns:
    s = scen.replace(f'_{tcgi}','')
    if s.startswith('base_'):
        s = 'base'
    new_cols.append((s, stat))
df.columns = pd.MultiIndex.from_tuples(new_cols, names=['Scenario','Statistic'])
df = df.loc[:, ~df.columns.duplicated()]

# --- 3. Compute “mean (min–max)” for each scenario ---
summary = pd.DataFrame(index=df.index)
for scen in df.columns.levels[0]:
    mins = df[(scen,'min')]
    maxs = df[(scen,'max')]
    meds = df[(scen,'median')]
    summary[scen] = [
        f"{m:.2f} ({mn:.2f}-{mx:.2f})"
        for m, mn, mx in zip(meds, mins, maxs)
    ]

# --- 4. Load ERA-5 rp_100 and insert as first column “ERA5” ---
hazard_dir = SYSTEM_DIR / "hazard" / "future" / "CHAZ_update" / "maps"
int_path   = hazard_dir / "TC_global_0300as_CHAZ_ERA5_exceedance_intensity.nc"
ds_int     = xr.open_dataset(int_path)

cities = {
    "Noumea":    {"lon": 166.45,  "lat": -22.28},
    "Miami":     {"lon": -80.2,   "lat": 25.8},
    "Acapulco":  {"lon": -99.9,   "lat": 16.8},
    "Toamasina": {"lon": 49.37,   "lat": -18.15},
    "Mumbai":    {"lon": 72.8,    "lat": 18.9},
    "Manila":    {"lon": 121.0,   "lat": 14.6},
}

lons = ds_int.lon.values
lats = ds_int.lat.values

def nearest_idx(lon_t, lat_t):
    # handle 0–360 grids
    if lons.min() >= 0 and lon_t < 0:
        lon_t += 360
    d2 = (lons - lon_t)**2 + (lats - lat_t)**2
    return int(np.argmin(d2))

# build ERA5 series
era5 = {}
for city, crd in cities.items():
    idx = nearest_idx(crd["lon"], crd["lat"])
    era5_val = float(ds_int["rp_100"].values[idx])
    era5[city] = round(era5_val, 2)

era5_ser = pd.Series(era5, name="ERA5")
# insert as first column (format to two decimals)
summary.insert(0, "ERA5", era5_ser[summary.index].map(lambda x: f"{x:.2f}"))

# --- 5. Save out ---
out_name = f'/Users/simonameiler/Documents/work/03_code/repos/CHAZ-hazard-maps/CHAZ-hazard-maps/data/rp100_summary_{tcgi}.csv'
summary.to_csv(out_name)
print(f"Wrote cleaned summary with ERA-5 to {out_name}")
print(summary)

Wrote cleaned summary with ERA-5 to /Users/simonameiler/Documents/work/03_code/repos/CHAZ-hazard-maps/CHAZ-hazard-maps/data/rp100_summary_SD.csv
            ERA5                 base          fut1_ssp245   
Noumea     52.96  54.08 (51.73-57.03)  52.91 (51.43-55.08)  \
Miami      51.44  50.84 (48.50-53.99)  50.83 (46.89-53.07)   
Acapulco   55.39  54.51 (51.05-56.02)  52.13 (46.24-56.44)   
Toamasina  53.92  56.09 (54.24-58.01)  54.24 (52.81-57.06)   
Mumbai     27.83  32.01 (29.62-34.34)  32.02 (24.67-37.19)   
Manila     51.59  52.23 (48.38-55.00)  53.98 (50.66-58.16)   

                   fut1_ssp370          fut1_ssp585          fut2_ssp245   
Noumea     53.20 (48.34-56.39)  53.15 (46.72-57.80)  53.09 (52.07-55.61)  \
Miami      49.70 (46.49-52.63)  50.66 (46.12-53.65)  47.48 (44.50-53.28)   
Acapulco   53.50 (45.74-59.18)  53.31 (48.82-56.36)  52.21 (40.97-55.66)   
Toamasina  53.52 (49.99-56.36)  52.28 (50.17-55.48)  53.44 (52.28-57.04)   
Mumbai     29.53 (27.58-31.26)  32.58 (2

### Table 4 and 5

Create summary table of Cat. 1 return periods estimates for ERA5, GCM-baseline, and GCM future runs for different emission scenarios (ssp245, ssp370, ssp585) and both future periods (fut1 = 2041-2060, fut2 = 2081-2100).

In [ ]:
def find_nearest(lons, lats, lon0, lat0):
    """Return the flat index of the nearest point on a 1D lon/lat grid."""
    # handle 0–360 grids
    if lons.min() >= 0 and lon0 < 0:
        lon0 += 360
    d2 = (lons - lon0)**2 + (lats - lat0)**2
    return int(np.argmin(d2))

tcgi = "CRH"  # or "SD"

data_dir = Path('/Users/simonameiler/Documents/work/03_code/repos/CHAZ-hazard-maps/CHAZ-hazard-maps/data/')

# 1) Read raw GCM min/max table
in_csv = Path(f"thresholds_gcm_minmaxmed_{tcgi}.csv")
df = pd.read_csv(data_dir/in_csv, index_col=0, header=[0,1,2])

# 2) Extract only thr_33 columns, strip suffix, collapse base_*
thr = "thr_33"
# select all columns where second level == thr_33
df33 = df.xs(thr, axis=1, level=1)
# df33 now has MultiIndex cols (Scenario, Statistic)
new_cols = []
for scen, stat in df33.columns:
    s = scen.replace(f"_{tcgi}", "")
    if s.startswith("base_"):
        s = "base"
    new_cols.append((s, stat))
df33.columns = pd.MultiIndex.from_tuples(new_cols, names=["Scenario","Statistic"])
# drop duplicate base
df33 = df33.loc[:, ~df33.columns.duplicated()]

# 3) Build the mean(min–max) summary
summary = pd.DataFrame(index=df33.index)
for scen in df33.columns.levels[0]:
    mn = df33[(scen, "min")]
    mx = df33[(scen, "max")]
    mean = (mn + mx) / 2
    # format with one decimal
    summary[scen] = [
        f"{m:.1f} ({low:.1f}-{high:.1f})"
        for m, low, high in zip(mean, mn, mx)
    ]

# 4) Add ERA-5 thr_33 from the return_periods file
hazard_dir = SYSTEM_DIR / "hazard" / "future" / "CHAZ_update" / "maps"
ds_rp = xr.open_dataset(hazard_dir / "TC_global_0300as_CHAZ_ERA5_return_periods.nc")

# define your cities in the same order
cities = {
    "Noumea":    (166.45, -22.28),
    "Miami":     (-80.2,   25.8),
    "Acapulco":  (-99.9,   16.8),
    "Toamasina": (49.37,  -18.15),
    "Mumbai":    (72.8,    18.9),
    "Manila":    (121.0,   14.6),
}

lons = ds_rp.lon.values
lats = ds_rp.lat.values

era5 = {}
for city, (lon, lat) in cities.items():
    idx = find_nearest(lons, lats, lon, lat)
    val = float(ds_rp["thr_33"].values[idx])
    era5[city] = round(val, 1)

era5_ser = pd.Series(era5, name="ERA5")
# insert as first column
summary.insert(0, "ERA5", era5_ser[summary.index].map(lambda x: f"{x:.1f}"))

# 5) Save
out_csv = data_dir / f"thresholds_summary_thr33_{tcgi}.csv"
summary.to_csv(out_csv)
print(f"Wrote thr_33 summary → {out_csv}")
print(summary)

Wrote thr_33 summary → /Users/simonameiler/Documents/work/03_code/repos/CHAZ-hazard-maps/CHAZ-hazard-maps/data/thresholds_summary_thr33_CRH.csv
            ERA5                base         fut1_ssp245         fut1_ssp370   
Noumea       9.5       7.6 (6.3-8.8)       7.2 (5.8-8.5)       7.4 (6.2-8.6)  \
Miami        8.7      8.7 (6.9-10.5)       8.3 (6.9-9.8)      7.4 (4.1-10.7)   
Acapulco     4.7       5.5 (4.0-6.9)       5.5 (3.5-7.5)      8.5 (4.3-12.7)   
Toamasina    7.8       6.4 (5.4-7.3)       5.8 (4.3-7.2)   368.8 (4.3-733.3)   
Mumbai     229.1  131.7 (88.5-174.8)  154.2 (84.1-224.3)  120.4 (82.8-157.9)   
Manila       6.6       6.2 (5.2-7.1)       5.6 (5.1-6.0)       6.0 (4.4-7.6)   

                  fut1_ssp585         fut2_ssp245         fut2_ssp370   
Noumea          7.3 (6.2-8.4)       7.4 (5.7-9.1)      8.2 (5.9-10.5)  \
Miami          7.8 (5.1-10.5)      9.7 (5.5-13.9)     10.7 (5.8-15.7)   
Acapulco       7.7 (3.9-11.5)      7.9 (4.3-11.5)      8.9 (3.5-14.4)   
Toa